## Data import

In [76]:
import pandas as pd
import numpy as np
import warnings

data_path = '../data/competitive-data-science-predict-future-sales/'

warnings.filterwarnings(action='ignore')

sales_train = pd.read_csv(data_path + 'sales_train.csv')
shops = pd.read_csv(data_path + 'shops.csv')
items = pd.read_csv(data_path + 'items.csv')
item_categories = pd.read_csv(data_path + 'item_categories.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [77]:
sales_train = sales_train.rename(columns={'date': '날짜',
                                          'date_block_num': '월ID',
                                          'shop_id': '상점ID',
                                          'item_id': '상품ID',
                                          'item_price': '판매가',
                                          'item_cnt_day': '판매량'})

shops = shops.rename(columns={'shop_name': '상점명',
                              'shop_id': '상점ID'})

items = items.rename(columns={'item_name': '상품명',
                              'item_id': '상품ID',
                              'item_category_id': '상품분류ID'})

item_categories = item_categories.rename(columns={
    'item_category_name': '상품분류명',
    'item_category_id': '상품분류ID',
})

test = test.rename(columns={'shop_id': '상점ID',
                            'item_id': '상품ID'})

In [78]:
def downcast(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')

    end_mem = df.memory_usage().sum() / 1024 ** 2

    if verbose:
        print("{:.1f}% 압축됨".format(100 * (start_mem - end_mem) / start_mem))

    return df

In [79]:
all_df = [sales_train, shops, items, item_categories, test]
for df in all_df:
    df = downcast(df)

54.2% 압축됨
38.5% 압축됨
54.2% 압축됨
39.8% 압축됨
70.8% 압축됨


## 데이터 전처리

### sales_train

In [80]:
# 이상치 제거
sales_train = sales_train[sales_train['판매가'] > 0]
sales_train = sales_train[sales_train['판매가'] < 50000]
sales_train = sales_train[sales_train['판매량'] > 0]
sales_train = sales_train[sales_train['판매량'] < 1000]

In [81]:
# 중복상점명 제거 -> 예시

print(shops['상점명'][0], '||', shops['상점명'][57])

!Якутск Орджоникидзе, 56 фран || Якутск Орджоникидзе, 56


In [82]:
sales_train.loc[sales_train['상점ID'] == 0, '상점ID'] = 57
sales_train.loc[sales_train['상점ID'] == 1, '상점ID'] = 58
sales_train.loc[sales_train['상점ID'] == 10, '상점ID'] = 11
sales_train.loc[sales_train['상점ID'] == 39, '상점ID'] = 40

test.loc[test['상점ID'] == 0, '상점ID'] = 57
test.loc[test['상점ID'] == 1, '상점ID'] = 58
test.loc[test['상점ID'] == 10, '상점ID'] = 11
test.loc[test['상점ID'] == 39, '상점ID'] = 40

In [83]:
shops['도시'] = shops['상점명'].apply(lambda x: x.split()[0])
shops['도시'].unique()

array(['!Якутск', 'Адыгея', 'Балашиха', 'Волжский', 'Вологда', 'Воронеж',
       'Выездная', 'Жуковский', 'Интернет-магазин', 'Казань', 'Калуга',
       'Коломна', 'Красноярск', 'Курск', 'Москва', 'Мытищи', 'Н.Новгород',
       'Новосибирск', 'Омск', 'РостовНаДону', 'СПб', 'Самара', 'Сергиев',
       'Сургут', 'Томск', 'Тюмень', 'Уфа', 'Химки', 'Цифровой', 'Чехов',
       'Якутск', 'Ярославль'], dtype=object)

In [84]:
shops.loc[shops['도시'] == '!Якутск', '도시'] = 'Якутск'

### shops

In [85]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
shops['도시'] = label_encoder.fit_transform(shops['도시'])

In [86]:
shops = shops.drop('상점명', axis = 1)
shops.head()

,상점ID,도시
0,0,29
1,1,29
2,2,0
3,3,1
4,4,2


### items

In [87]:
items = items.drop(['상품명'], axis = 1)
items['첫 판매월'] = sales_train.groupby('상품ID'). agg({'월ID': 'min'})['월ID']

items.head()

,상품ID,상품분류ID,첫 판매월
0,0,40,20.0
1,1,76,15.0
2,2,40,19.0
3,3,40,18.0
4,4,40,20.0


In [88]:
items[items['첫 판매월'].isna()]

,상품ID,상품분류ID,첫 판매월
83,83,40,NaN
140,140,45,NaN
168,168,44,NaN
173,173,45,NaN
204,204,44,NaN
...,...,...,...
21974,21974,61,NaN
21975,21975,61,NaN
22022,22022,40,NaN
22035,22035,40,NaN


In [89]:
# 결측치는 마지막날의 데이터
items['첫 판매월'] = items['첫 판매월'].fillna(34)

### item_categories

In [90]:
item_categories['대분류'] = item_categories['상품분류명'].apply(lambda x: x.split()[0])

In [91]:
item_categories['대분류'].value_counts()

대분류
Игры          14
Книги         13
Подарки       12
Игровые        8
Аксессуары     7
Музыка         6
Программы      6
Карты          5
Кино           5
Служебные      2
Чистые         2
PC             1
Билеты         1
Доставка       1
Элементы       1
Name: count, dtype: int64

In [92]:
# apply 함수의 변형
def make_etc(x):
    if len(item_categories[item_categories['대분류']==x]) >= 5:
        return x
    else:
        return 'etc'
    
item_categories['대분류'] = item_categories['대분류'].apply(make_etc)

In [93]:
label_encoder = LabelEncoder()

item_categories['대분류'] = \
label_encoder.fit_transform(item_categories['대분류'])

In [94]:
item_categories = item_categories.drop('상품분류명', axis=1)

## 데이터 조합 및 파생 피쳐

### merge

In [95]:
from itertools import product

train = []

for i in sales_train['월ID'].unique():
    all_shop = sales_train.loc[sales_train['월ID'] == i, '상점ID'].unique()
    all_item = sales_train.loc[sales_train['월ID'] == i, '상품ID'].unique()
    train.append(np.array(list(product([i], all_shop, all_item))))

idx_features = ['월ID', '상점ID', '상품ID']
train = pd.DataFrame(np.vstack(train), columns= idx_features)

In [96]:
group = sales_train.groupby(idx_features).agg({'판매량': 'sum',
                                               '판매가': 'mean'})

group = group.reset_index()
group = group.rename(columns={'판매량': '월간판매량', '판매가': '평균 판매가'})
train = train.merge(group, on=idx_features, how='left')

train.head()

,월ID,상점ID,상품ID,월간판매량,평균 판매가
0,0,59,22154,1.0,999.0
1,0,59,2552,NaN,NaN
2,0,59,2554,NaN,NaN
3,0,59,2555,NaN,NaN
4,0,59,2564,NaN,NaN


In [97]:
import gc
del group
gc.collect()

187

In [98]:
group = sales_train.groupby(idx_features).agg({'판매량': 'count'})
group = group.reset_index()
group = group.rename(columns={'판매량':'판매건수'})

train = train.merge(group, on=idx_features, how='left')

del group, sales_train
gc.collect()

train.head()

,월ID,상점ID,상품ID,월간판매량,평균 판매가,판매건수
0,0,59,22154,1.0,999.0,1.0
1,0,59,2552,NaN,NaN,NaN
2,0,59,2554,NaN,NaN,NaN
3,0,59,2555,NaN,NaN,NaN
4,0,59,2564,NaN,NaN,NaN


In [99]:
test['월ID'] = 34

all_data = pd.concat([train, test.drop('ID', axis=1)],
                     ignore_index=True,
                     keys=idx_features)

all_data = all_data.fillna(0)

all_data.head()

,월ID,상점ID,상품ID,월간판매량,평균 판매가,판매건수
0,0,59,22154,1.0,999.0,1.0
1,0,59,2552,0.0,0.0,0.0
2,0,59,2554,0.0,0.0,0.0
3,0,59,2555,0.0,0.0,0.0
4,0,59,2564,0.0,0.0,0.0


In [100]:
all_data = all_data.merge(shops, on='상점ID', how='left')
all_data = all_data.merge(items, on='상품ID', how='left')
all_data = all_data.merge(item_categories, on='상품분류ID', how='left')

all_data = downcast(all_data)

68.9% 압축됨


In [101]:
del shops, items, item_categories
gc.collect()

0

### 시차 피처 생성

In [102]:
def add_mean_features(df, mean_features, idx_features):
    assert (idx_features[0] == '월ID') and \
    len(idx_features) in [2,3] # 기준 피쳐 개수 확인

    # 파생피쳐명 생성
    if len(idx_features) == 2:
        feature_name  = idx_features[1] + "별 평균 판매량"
    else:
        feature_name = idx_features[1] + " " + idx_features[2] + '별 평균 판매량'

    # 기준 피쳐 기준 그룹화
    group = df.groupby(idx_features).agg({'월간판매량':'mean'})
    group = group.reset_index()
    group = group.rename(columns={'월간판매량': feature_name})

    # df와 group merge
    df = df.merge(group, on=idx_features, how='left')
    df = downcast(df, verbose=False)

    mean_features.append(feature_name)

    # 가비지 컬렉션
    del group
    gc.collect()

    return df, mean_features


In [103]:
item_mean_features = []

all_data, item_mean_features = add_mean_features(all_data,
                                                 mean_features=item_mean_features,
                                                 idx_features=['월ID', '상품ID'])

all_data, item_mean_features = add_mean_features(all_data,
                                                 mean_features=item_mean_features,
                                                 idx_features=['월ID', '상품ID', '도시'])

In [104]:
shop_mean_features = []

all_data, shop_mean_features = add_mean_features(all_data,
                                                 mean_features=shop_mean_features,
                                                 idx_features=['월ID', '상점ID', '상품분류ID'])

In [105]:
item_mean_features

['상품ID별 평균 판매량', '상품ID 도시별 평균 판매량']

In [106]:
shop_mean_features

['상점ID 상품분류ID별 평균 판매량']

In [109]:
def add_lag_features(df, lag_features_to_clip,
                     idx_features, lag_feature, nlags=3, clip=False):
    missing_cols = [col for col in idx_features + [lag_feature] if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing columns in the dataframe: {missing_cols}")

    df_temp = df[idx_features + [lag_feature]].copy()

    for i in range(1, nlags + 1):
        lag_feature_name = lag_feature + "_시차" + str(i)
        df_temp.columns = idx_features + [lag_feature_name]
        df_temp['월ID'] += 1
        df = df.merge(df_temp.drop_duplicates(), on=idx_features, how='left')

        df[lag_feature_name] = df[lag_feature_name].fillna(0)
        if clip:
            lag_features_to_clip.append(lag_feature_name)
        df = downcast(df, False)

    del df_temp
    gc.collect()

    return df, lag_features_to_clip

In [110]:
lag_features_to_clip = []
idx_features = ['월ID', '상점ID', '상품ID']

all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='월간판매량',
                                                  nlags=3,
                                                  clip=True)

In [111]:
all_data.head().T

,0,1,2,3,4
월ID,0.000000,0.000000,0.000000,0.000000,0.000000
상점ID,59.000000,59.000000,59.000000,59.000000,59.000000
상품ID,22154.000000,2552.000000,2554.000000,2555.000000,2564.000000
월간판매량,1.000000,0.000000,0.000000,0.000000,0.000000
평균 판매가,999.000000,0.000000,0.000000,0.000000,0.000000
판매건수,1.000000,0.000000,0.000000,0.000000,0.000000
도시,30.000000,30.000000,30.000000,30.000000,30.000000
상품분류ID,37.000000,58.000000,58.000000,56.000000,59.000000
첫 판매월,0.000000,0.000000,0.000000,0.000000,0.000000
대분류,5.000000,7.000000,7.000000,7.000000,7.000000


In [112]:
lag_features_to_clip

['월간판매량_시차1', '월간판매량_시차2', '월간판매량_시차3']

In [113]:
all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='판매건수',
                                                  nlags=3,
                                                  clip=True)

all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='평균 판매가',
                                                  nlags=3,
                                                  clip=True)

In [114]:
for item_mean_feature in item_mean_features:
    all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                    lag_features_to_clip=lag_features_to_clip,
                                                    idx_features=idx_features,
                                                    lag_feature=item_mean_feature,
                                                    nlags=3,
                                                    clip=True)
all_data = all_data.drop(item_mean_features, axis = 1)

In [115]:
for shop_mean_feature in shop_mean_features:
    all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                    lag_features_to_clip=lag_features_to_clip,
                                                    idx_features=['월ID', '상점ID', '상품분류ID'],
                                                    lag_feature=shop_mean_feature,
                                                    nlags=3,
                                                    clip=True)
all_data = all_data.drop(shop_mean_features, axis = 1)

In [116]:
all_data = all_data.drop(all_data[all_data['월ID'] < 3].index)

In [117]:
all_data.head().T

,1122386,1122387,1122388,1122389,1122390
월ID,3.000000,3.000000,3.000000,3.000000,3.000000
상점ID,25.000000,25.000000,25.000000,25.000000,25.000000
상품ID,8092.000000,7850.000000,8051.000000,8088.000000,8089.000000
월간판매량,3.000000,3.000000,3.000000,1.000000,1.000000
평균 판매가,149.000000,199.000000,30.000000,199.000000,199.000000
판매건수,3.000000,3.000000,2.000000,1.000000,1.000000
도시,13.000000,13.000000,13.000000,13.000000,13.000000
상품분류ID,40.000000,30.000000,66.000000,55.000000,55.000000
첫 판매월,0.000000,0.000000,0.000000,0.000000,0.000000
대분류,5.000000,3.000000,8.000000,7.000000,7.000000


### 기타 피쳐엔지니어링

In [118]:
all_data['월간판매량 시차평균'] = all_data[['월간판매량_시차1', '월간판매량_시차2', '월간판매량_시차3']].mean(axis=1)

In [119]:
all_data[lag_features_to_clip + ['월간판매량', '월간판매량 시차평균']] = \
all_data[lag_features_to_clip + ['월간판매량', '월간판매량 시차평균']].clip(0, 20)

In [120]:
all_data['시차변화량1'] = all_data['월간판매량_시차1'] / all_data['월간판매량_시차2']
all_data['시차변화량1'] = all_data['시차변화량1'].replace([np.inf, -np.inf], np.nan).fillna(0)

all_data['시차변화량2'] = all_data['월간판매량_시차2'] / all_data['월간판매량_시차3']
all_data['시차변화량2'] = all_data['시차변화량2'].replace([np.inf, -np.inf], np.nan).fillna(0)


In [121]:
all_data['신상여부'] = (all_data['첫 판매월'] == all_data['월ID'])

In [122]:
all_data['첫 판매 후 기간'] = all_data['월ID'] - all_data['첫 판매월']

In [123]:
all_data['월'] = all_data['월ID'] % 12

In [124]:
all_data = all_data.drop(['첫 판매월', '평균 판매가', '판매건수'], axis = 1)
all_data = downcast(all_data, False)

In [125]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9904582 entries, 1122386 to 11026967
Data columns (total 31 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   월ID                      int8   
 1   상점ID                     int8   
 2   상품ID                     int16  
 3   월간판매량                    int8   
 4   도시                       int8   
 5   상품분류ID                   int8   
 6   대분류                      int8   
 7   월간판매량_시차1                int8   
 8   월간판매량_시차2                int8   
 9   월간판매량_시차3                int8   
 10  판매건수_시차1                 int8   
 11  판매건수_시차2                 int8   
 12  판매건수_시차3                 int8   
 13  평균 판매가_시차1               float32
 14  평균 판매가_시차2               float32
 15  평균 판매가_시차3               float32
 16  상품ID별 평균 판매량_시차1         float32
 17  상품ID별 평균 판매량_시차2         float32
 18  상품ID별 평균 판매량_시차3         float32
 19  상품ID 도시별 평균 판매량_시차1      float32
 20  상품ID 도시별 평균 판매량_시차2      float32
 21  상

In [126]:
X_train = all_data[all_data['월ID'] < 33]
X_train = X_train.drop(['월간판매량'], axis=1)

X_valid = all_data[all_data['월ID'] == 33]
X_valid = X_valid.drop(['월간판매량'], axis=1)

X_test = all_data[all_data['월ID'] ==  34]
X_test = X_test.drop(['월간판매량'], axis=1)

y_train = all_data[all_data['월ID'] < 33]['월간판매량']

y_valid = all_data[all_data['월ID'] == 33]['월간판매량']


del all_data
gc.collect()

0

In [127]:
import lightgbm as lgb

params = {
    'metric': 'rmse',
    'num_leaves': 255,
    'learning_rate': 0.01,
    'force_col_wise': True,
    'random_state': 10
}

cat_features = ['상점ID', '상품분류ID']

dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)

lgb_model = lgb.train(params=params,
                      train_set=dtrain,
                      num_boost_round=500,
                      valid_sets=(dtrain, dvalid),
                      categorical_feature=cat_features)

preds = lgb_model.predict(X_test).clip(0,20)

submission['item_cnt_month'] = preds
submission.to_csv(data_path + 'submission.csv', index=False)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 3132
[LightGBM] [Info] Number of data points in the train set: 9452298, number of used features: 30
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 0.297707


In [265]:
del X_train, y_train, X_valid, y_valid, X_test, lgb_model, dvalid, dtrain
gc.collect()

472